# Queries and views for FHIR Data Pipes
This notebook includes examples for how to query data-warehouse files created
by FHIR Data Pipes. For the query engine, it is assumed that Spark SQL is
being used (see section below). But hopefully the SQL queries show the generic
pattern that can be used for any query engine that supports SQL on Parquet
files.

The main recommended pattern is to create simple and flat views out of the
complex nested/repeated schema that SQL-on-FHIR has. We first show how to do
this with SQL (Spark flavor of it). At the end, we also show how do to this with
[FHIR-views](https://github.com/google/fhir-py/tree/main/google-fhir-views).
Note that FHIR-views runner for Spark is still an experimental feature.

# Prerequisites:

**Note**: All commands need to be run from the root directory of the
fhir-data-pipes repo. We are setting up fhir-data-pipes controller and
a single node Spark process. The input synthetic data is uploaded to a local
HAPI server.

1. Bring up a Hive ThriftServer:
    ```
    docker-compose  -f ./docker/compose-controller-spark-sql-single.yaml up --build  --force-recreate -d
    ```

2. Bring up a HAPI FHIR server:
    ```
    docker-compose  -f ./docker/hapi-compose.yml up --force-recreate -d
    ```

3. Load data into the HAPI FHIR server:
    ```
    python3 ./synthea-hiv/uploader/main.py HAPI http://localhost:8091/fhir --input_dir ./synthea-hiv/sample_data
    ```

4. In the FHIR Pipelines Control Panel page, http://localhost:8090, click on the
   “Run Full” button to convert the FHIR data to Parquet files with SQL-on-FHIR
   schema. These files will be stored under the `docker/dwh` directory.


# Installation

**NOTE: If you are running this notebook inside the corresponding docker
container, you can skip the installation section as the requirements are
already included in that docker image.**

It's recommended to set up a virtual environment before starting the notebook
kernel to avoid any dependency version issues with your native environment.

This can be done with the following commands in a terminal, if you have Conda
installed (you can also use `virtualenv` instead of Conda):

```
    conda create -n python310 python=3.10 -y
    conda activate python310
    conda install ipykernel -y
    ipython kernel install --user --name=python310
    pip install -r requirements.txt
```

# Environment setup
The cell below sets up a Spark client and creates a Spark view "runner",
which is used to apply declarative views of FHIR in Spark. Note you may
need to do adjustments for the Thrift Sever address (`hive://`).
The value below assumes you are running this notebook in a docker container
on the same network as [this config](https://github.com/google/fhir-data-pipes/blob/3694f1394d0b9011ab480ca61f0bd0568bca2f53/docker/compose-controller-spark-sql-single.yaml#L71).

In [1]:
import pandas

from sqlalchemy import dialects
from sqlalchemy import engine

dialects.registry.register("hive", "pyhive.sqlalchemy_hive", "HiveDialect")

# The endpoint of the Hive ThriftServer to connect to; you may need to
# adjust this if you are not running this through the default docker container.
#query_engine = engine.create_engine("hive://localhost:10001/default")
query_engine = engine.create_engine("hive://spark-thriftserver:10000/default")

The cell below loads the parquet files we've created into our Spark ThriftServer.

**Note 1**: These tables are automatically created if `createHiveResourceTables`
is set to `true` like
[here](https://github.com/google/fhir-data-pipes/blob/3694f1394d0b9011ab480ca61f0bd0568bca2f53/docker/config/application.yaml#L34).

**Note 2**: If you choose to run the following cell and recreate the tables,
you probably need to override `destination_directory_path` to point to the
specific path of the last pipeline run, e.g.,
`/dwh/controller_DWH_TIMESTAMP_2023_09_07T18_11_10_988888411Z`.
The default value below, i.e., `/dwh/controller_*`, assigns _all_ Parquet files
in _all_ of such directories into a _single_ table for each resource.

**Note 3**: The full name of tables have a database name as well which is
`default` but default; so instead of `default.Patient` we simply use `Patient`.

In [2]:
# Please read the note above before recreating resource tables!

destination_directory_path = "/dwh/controller_*"
with query_engine.connect() as con:
    # The following lines are commented out to prevent unintentional deletion.
    # con.execute(f"DROP TABLE IF EXISTS Encounter;")
    # con.execute(f"DROP TABLE IF EXISTS Observation;")
    # con.execute(f"DROP TABLE IF EXISTS Patient;")
    con.execute(
        f"CREATE TABLE IF NOT EXISTS Encounter USING"
        f" PARQUET LOCATION '{destination_directory_path}/Encounter/*.parquet';"
    )
    con.execute(
        f"CREATE TABLE IF NOT EXISTS Observation USING PARQUET"
        f" LOCATION '{destination_directory_path}/Observation/*.parquet';"
    )
    con.execute(
        f"CREATE TABLE IF NOT EXISTS Patient USING PARQUET"
        f" LOCATION '{destination_directory_path}/Patient/*.parquet';"
    )

# Explore the data with SQL
At this point you should be able to run any SQL query against the resource
tables. The following subsections show some sample queries. If you want, you can
skip directly to the [FHIR-vewis section](#FHIR-Views).

In [3]:
pandas.read_sql_query(
    sql=f"SELECT COUNT(*) FROM Patient",
    con=query_engine,
)

,count(1)
0,79


In [4]:
pandas.read_sql_query(
    sql=(f"SELECT COUNT(*) FROM Observation AS O"
      " WHERE O.effective.DateTime > '2010-01-01'"),
    con=query_engine,
)

,count(1)
0,5680


## Exploring Observation codes
The [SQL-on-FHIR schema](https://github.com/FHIR/sql-on-fhir/blob/master/sql-on-fhir.md)
resembles the JSON structure of FHIR resources and hence has many nested and
repeated structures. It usually makes sense to flatten the columns we need to
deal with. This flattening process varies depending on the SQL dialect.
Here is an example for Observation codes.

_Note_: The actual codes and systems should be ignored as this is for a
synthetic datasets that has not gone through concept mapping.

In [5]:
pandas.read_sql_query(
    sql="""
      SELECT OCC.`system` AS code_sys, OCC.code, OCC.display,
        COUNT(*) AS num_obs, AVG(O.value.quantity.value) AS avg_val
      FROM Observation AS O LATERAL VIEW explode(code.coding) AS OCC
      GROUP BY OCC.`system`, OCC.code, OCC.display
      ORDER BY num_obs DESC
      LIMIT 20; """,
    con=query_engine,
)

,code_sys,code,display,num_obs,avg_val
0,http://loinc.org,1271AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TESTS ORDERED,4949,NaN
1,http://loinc.org,1088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,CURRENT ANTIRETROVIRAL DRUGS USED FOR TREATMENT,2387,NaN
2,http://loinc.org,1111AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,PATIENT REPORTED CURRENT TB TREATMENT,2043,NaN
3,http://loinc.org,1250AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,ANTIRETROVIRALS STARTED,1805,NaN
4,http://loinc.org,1270AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TUBERCULOSIS TREATMENT STARTED,1121,NaN
5,http://loinc.org,159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,REVIEW OF TUBERCULOSIS SCREENING QUESTIONS,738,NaN
6,http://loinc.org,5085AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,LOINC Code,530,163.291043
7,http://loinc.org,159911AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,PATIENT REPORTED CURRENT ANTIRETROVIRAL TREATMENT,408,NaN
8,http://loinc.org,1261AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,PCP PROPHYLAXIS PLAN,340,NaN
9,http://loinc.org,1265AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TUBERCULOSIS PROPHYLAXIS PLAN,340,NaN


## Indicator example
The following query counts number of patients that have had an observation
with a specific code (HIV viral load), with a value below a certain threshold
(400000), during a specific reporting period (year 2010 in this example).
This is a useful pattern in many cases, e.g., calculating TX_CURR or TX_PVLS
indicators of [PEPFAR](https://www.state.gov/pepfar-fy-2023-mer-indicators/)
(for TX_CURR we need to look at `O.value.codeableConcept.coding`).

In [6]:
pandas.read_sql_query(
    sql="""
      SELECT COUNT(DISTINCT O.subject.PatientId) AS num_patients
      FROM Observation AS O LATERAL VIEW explode(code.coding) AS OCC
      WHERE OCC.code LIKE '856%%'
        AND OCC.`system` = 'http://loinc.org'
        AND O.value.quantity.value < 400000
        AND YEAR(O.effective.dateTime) = 2010; """,
    con=query_engine,
)

,num_patients
0,5


## Creating flat views
It is usually helpful to create flat views and build other queries
on top of those views. Here is an example for Observation:

In [7]:
with query_engine.connect() as con:
    con.execute("""
      CREATE OR REPLACE VIEW flat_observation AS
      SELECT O.id AS obs_id, O.subject.PatientId AS patient_id,
        OCC.`system` AS code_sys, OCC.code,
        O.value.quantity.value AS val_quantity,
        OVCC.code AS val_code, OVCC.`system` AS val_sys,
        O.effective.dateTime AS obs_date
      FROM Observation AS O LATERAL VIEW OUTER explode(code.coding) AS OCC
        LATERAL VIEW OUTER explode(O.value.codeableConcept.coding) AS OVCC
      ;
      """
    )

pandas.read_sql_query(
    sql="SELECT * FROM flat_observation LIMIT 5;",
    con=query_engine,
)

,obs_id,patient_id,code_sys,code,val_quantity,val_code,val_sys,obs_date
0,10030,9040,http://loinc.org,159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN,140238AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,http://snomed.info/sct,2009-12-19T03:22:24+00:00
1,10063,9040,http://loinc.org,5088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,36.921,None,None,2011-03-21T03:22:24+00:00
2,10075,9040,http://loinc.org,1271AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN,1107AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,http://snomed.info/sct,2011-03-21T03:22:24+00:00
3,10083,9040,http://loinc.org,1271AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN,305AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,http://snomed.info/sct,2011-03-21T03:22:24+00:00
4,10197,9040,http://loinc.org,159911AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN,1652AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,http://snomed.info/sct,2013-04-30T03:22:24+00:00


Repeating the same TX_CURR query with the new view is much more readable:

In [8]:
pandas.read_sql_query(
    sql="""SELECT COUNT(DISTINCT patient_id) AS num_patients
      FROM flat_observation
      WHERE code LIKE '856%%'
        AND code_sys = 'http://loinc.org'
        AND val_quantity < 400000
        AND YEAR(obs_date) = 2010
      LIMIT 100; """,
    con=query_engine,
)

,num_patients
0,5


# FHIR Views

**Note**: The Spark-runner for FHIR-views is in experimental mode and is not
ready for production use.

[FHIR Views](https://github.com/google/fhir-py/tree/main/google-fhir-views)
is abstraction layer to simplify the above view creation pattern. It helps
creating flat views for FHIR resources using
[FHIRPath](http://hl7.org/fhirpath/N1/) statements. It separates
view definition from view creation, the latter being dependent on the
underlying data representation. For example, there is a BigQuery-runner,
Spark-runner, etc. each of which transforms the view definition into the
corresponding SQL dialect.

The FHIR-Views pattern is being standardized in
[SQL-on-FHIR v2](https://build.fhir.org/ig/FHIR/sql-on-fhir-v2/)
as a language independent FHIR spec (i.e., JSON instead of Python).

In [9]:
# In addition to the sqlalchemy imports we did above, we also need the
# FHIR-views library for the rest of this notebook.

from google.fhir.views import r4
from google.fhir.views import spark_runner

# The Spark dataset containing FHIR data. This may be read-only to the user.
fhir_dataset = "default"

# The Spark dataset where we will create views, value sets, and other derived tables
# as needed. This must be writeable by the user.
analysis_dataset = "demo_example"

# Create a runner to execute the views over Spark.
runner = spark_runner.SparkRunner(
    query_engine=query_engine,
    fhir_dataset=fhir_dataset,
    view_dataset=analysis_dataset,
    snake_case_resource_tables=True,
)

## Explore the data with FHIR Views
Similar to the SQL examples above, first we try to get a general feeling of
the data. So let's create some views of FHIR resources and take a look at
common fields.

In [10]:
# Load views based on the base FHIR R4 profile definitions.
views = r4.base_r4()

# Create shorthand names for resources we will work with.
obs = views.view_of("Observation")
pats = views.view_of("Patient")

Let's take a look at some observation resources by creating FHIRPath expressions to select the items we're interested in.

Notice that tab completion works for FHIR fields and FHIRPath functions, so users don't need to switch back and forth to FHIR documentation as much.

In [11]:
runner.to_dataframe(
    obs.select(
        {
            "id": obs.id,
            "category": obs.category.coding.code,
            "code_display": obs.code.coding.display,
            "status": obs.status,
        }
    ),
    limit=10,
)

,id,category,code_display,status
0,10929,"[""survey""]","[""CURRENT ANTIRETROVIRAL DRUGS USED FOR TREATM...",final
1,10063,"[""survey""]","[""TEMPERATURE""]",final
2,10798,"[""survey""]","[""TEMPERATURE""]",final
3,10030,"[""survey""]","[""REVIEW OF TUBERCULOSIS SCREENING QUESTIONS""]",final
4,10803,"[""survey""]","[""REVIEW OF TUBERCULOSIS SCREENING QUESTIONS""]",final
5,10880,"[""survey""]","[""PATIENT REPORTED CURRENT TB TREATMENT""]",final
6,10075,"[""survey""]","[""TESTS ORDERED""]",final
7,10083,"[""survey""]","[""TESTS ORDERED""]",final
8,10825,"[""survey""]","[""TESTS ORDERED""]",final
9,10197,"[""survey""]","[""PATIENT REPORTED CURRENT ANTIRETROVIRAL TREA...",final


Of course just a sample of codes isn't too useful -- we really want a summary of what codes exist in a field and how many there are. Fortunately, the Spark runner supports a `summarize_codes` method that accepts a view and a field name and does exactly that. Note that we needed a non-trivial SQL query to do something similar.

In [12]:
runner.summarize_codes(obs, obs.code)

,system,code,display,count
0,http://loinc.org,1271AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TESTS ORDERED,4949
1,http://loinc.org,1088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,CURRENT ANTIRETROVIRAL DRUGS USED FOR TREATMENT,2387
2,http://loinc.org,1111AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,PATIENT REPORTED CURRENT TB TREATMENT,2043
3,http://loinc.org,1250AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,ANTIRETROVIRALS STARTED,1805
4,http://loinc.org,1270AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TUBERCULOSIS TREATMENT STARTED,1121
5,http://loinc.org,159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,REVIEW OF TUBERCULOSIS SCREENING QUESTIONS,738
6,http://loinc.org,5085AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,LOINC Code,530
7,http://loinc.org,159911AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,PATIENT REPORTED CURRENT ANTIRETROVIRAL TREATMENT,408
8,http://loinc.org,1268AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TUBERCULOSIS TREATMENT PLAN,340
9,http://loinc.org,1265AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,TUBERCULOSIS PROPHYLAXIS PLAN,340


## Creating Views

As discussed above, to handle the complexity of FHIR data, we can usually create
a flat, tabular form of a resource type that can satisfy a given use case.

For example, imagine we need a simple table of patients with their current
address. This isn't trivial to query since the address is a nested repeated
field. Fortunately we can build a FHIRPath expression to find the current
address and create a flattened view using that.

(This can vary by dataset, but in this case we determine the current address
by finding the first address that does not have a period attached to it.)

In [13]:
# For this dataset we interpret the current address as one where period is empty.

current = pats.address.where(pats.address.period.empty())

simple_pats = pats.select(
    {
        "id": pats.id,
        "gender": pats.gender,
        "birthdate": pats.birthDate,
        "street": current.line,
        "city": current.city,
        "state": current.state,
        "zip": current.postalCode,
    }
)

runner.to_dataframe(simple_pats, limit=5)

,id,gender,birthdate,street,city,state,zip
0,38462,female,1961-10-01,"[""376 Mohr Annex Suite 88""]","[""Northampton""]","[""MA""]","[""01060""]"
1,9040,female,1957-10-31,"[""751 Kessler Divide Unit 49""]","[""Brookline""]","[""MA""]","[""02445""]"
2,39570,male,1915-04-20,"[""988 Ryan Burg Apt 59""]","[""North Reading""]","[""MA""]",[]
3,27113,male,1962-04-23,"[""609 Steuber Crossroad Unit 49""]","[""Westfield""]","[""MA""]","[""01085""]"
4,30404,male,1941-10-24,"[""617 D'Amore Course""]","[""Brewster""]","[""MA""]","[""02631""]"


That's nice, but suppose we want to create it as an actual Spark view, i.e.,
a virtual table that can be easily used by any application that uses Spark.
This is basically what we did in the previous SQL section using `CREATE VIEW`.
We can simply turn the above definition into a Spark/Hive view as well:

In [14]:
runner.create_database_view(simple_pats, "patient_current_address")

Now we can run simple SQL queries against this view.

_Note_: Since we set `view_dataset='demo_example'` when creating the Spark
runner in the [Environment setup section](#Environment-setup), we need to
include that in the SQL queries we write against these views.

In [15]:
pandas.read_sql_query(
    sql="SELECT * FROM demo_example.patient_current_address LIMIT 5",
    con=query_engine,
)

,id,gender,birthdate,street,city,state,zip
0,38462,female,1961-10-01,"[""376 Mohr Annex Suite 88""]","[""Northampton""]","[""MA""]","[""01060""]"
1,9040,female,1957-10-31,"[""751 Kessler Divide Unit 49""]","[""Brookline""]","[""MA""]","[""02445""]"
2,39570,male,1915-04-20,"[""988 Ryan Burg Apt 59""]","[""North Reading""]","[""MA""]",[]
3,27113,male,1962-04-23,"[""609 Steuber Crossroad Unit 49""]","[""Westfield""]","[""MA""]","[""01085""]"
4,30404,male,1941-10-24,"[""617 D'Amore Course""]","[""Brewster""]","[""MA""]","[""02631""]"


Now we have a nice, flattened patients table that meets the needs of our system and data.

## Observation view for indicators
Here we try to recreate a view for calculating some PEPFAR indicators.
This is similar to what we did with direct SQL above; but now we use
FHIR-Views for the heavy lifting.

In [16]:
# TODO: fix the `where` clause Spark SQL issues.

obs_code_val = obs.select(
    {
        "obs_id": obs.id,
        "patient_id": obs.subject.idFor('Patient'),
        "code_sys": obs.code.coding.system.first(),
        "code": obs.code.coding.code.first(),
        "val_quantity": obs.value.ofType('quantity').value,
    }).where(
        # obs.code.coding.system=='http://loinc.org'
        # obs.code.coding.code=='1088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
)

runner.to_dataframe(obs_code_val, limit=20)
#print(runner.to_sql(obs_code_val))

,obs_id,patient_id,code_sys,code,val_quantity
0,10929,10392,http://loinc.org,1088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN
1,11327,10669,http://loinc.org,1088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN
2,12026,11149,http://loinc.org,5085AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,188.660
3,10063,9040,http://loinc.org,5088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,36.921
4,10798,10392,http://loinc.org,5088AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,36.833
5,10030,9040,http://loinc.org,159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN
6,10803,10392,http://loinc.org,159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN
7,11995,11149,http://loinc.org,159800AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN
8,11554,10669,http://loinc.org,1261AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN
9,10880,10392,http://loinc.org,1111AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA,NaN


## Checking the underlying SQL statements
It is easy to expose the underlying SQL query that the FHIR-views library generates. This is done using the `to_sql` method that is part of the runner API. We show below the SQL queries generated of the two examples we used above:

In [17]:
print(runner.to_sql(
    obs.select(
        {
            "id": obs.id,
            "category": obs.category.coding.code,
            "code_display": obs.code.coding.display,
            "status": obs.status,
        }
    ),
    limit=10,
))

SELECT (SELECT id) AS id,(SELECT COLLECT_LIST(code)
FROM (SELECT coding_element_.code
FROM (SELECT category_element_
FROM (SELECT EXPLODE(category_element_) AS category_element_ FROM (SELECT category AS category_element_))) LATERAL VIEW POSEXPLODE(category_element_.coding) AS index_coding_element_, coding_element_)
WHERE code IS NOT NULL) AS category,(SELECT COLLECT_LIST(display)
FROM (SELECT coding_element_.display
FROM (SELECT code) LATERAL VIEW POSEXPLODE(code.coding) AS index_coding_element_, coding_element_)
WHERE display IS NOT NULL) AS code_display,(SELECT status) AS status,(SELECT subject.patientId AS idFor_) AS __patientId__ FROM `default`.observation LIMIT 10


In [18]:
current = pats.address.where(pats.address.period.empty())

simple_pats = pats.select(
    {
        "id": pats.id,
        "gender": pats.gender,
        "birthdate": pats.birthDate,
        "street": current.line,
        "city": current.city,
        "state": current.state,
        "zip": current.postalCode,
    }
)

print(runner.to_sql(simple_pats, limit=5))

SELECT (SELECT id) AS id,(SELECT gender) AS gender,(SELECT CAST(birthDate AS TIMESTAMP) AS birthDate) AS birthdate,(SELECT COLLECT_LIST(line_element_)
FROM (SELECT line_element_
FROM (SELECT address_element_
FROM (SELECT EXPLODE(address_element_) AS address_element_ FROM (SELECT address AS address_element_))
WHERE (SELECT CASE WHEN COUNT(*) = 0 THEN TRUE ELSE FALSE END AS empty_
FROM (SELECT address_element_.period)
WHERE period IS NOT NULL)) LATERAL VIEW POSEXPLODE(address_element_.line) AS index_line_element_, line_element_)
WHERE line_element_ IS NOT NULL) AS street,(SELECT COLLECT_LIST(city)
FROM (SELECT address_element_.city
FROM (SELECT EXPLODE(address_element_) AS address_element_ FROM (SELECT address AS address_element_))
WHERE (SELECT CASE WHEN COUNT(*) = 0 THEN TRUE ELSE FALSE END AS empty_
FROM (SELECT address_element_.period)
WHERE period IS NOT NULL))
WHERE city IS NOT NULL) AS city,(SELECT COLLECT_LIST(state)
FROM (SELECT address_element_.state
FROM (SELECT EXPLODE(addres